In [ ]:
# Check if installation was succesful.
try:
  print('Checking that the installation succeeded:')
  import mujoco
  mujoco.MjModel.from_xml_string('<mujoco/>')
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')

print('Installation successful.')

import time
import itertools
import numpy as np

# Graphics and plotting.
print('Installing mediapy:')
# !command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
# !pip install -q mediapy
import mediapy as media
import matplotlib.pyplot as plt

# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)

from IPython.display import clear_output
clear_output()

In [ ]:
from pathlib import Path
import enum
from tqdm import tqdm

class Resolution(enum.Enum):
  SD = (480, 640)
  HD = (720, 1280)
  UHD = (2160, 3840)

res = Resolution.SD
h, w = res.value

model_dir = Path("mujoco_menagerie/hello_robot_stretch") # You could also use google_robot
model_xml = model_dir / "scene.xml"

# Load model.
model = mujoco.MjModel.from_xml_path(str(model_xml))
data = mujoco.MjData(model)

# Make sure offscreen rendering can support the desired resolution.
model.vis.global_.offheight = h
model.vis.global_.offwidth = w

renderer = mujoco.Renderer(model, height=h, width=w)

mujoco.mj_forward(model, data)
renderer.update_scene(data)
media.show_image(renderer.render())

In [ ]:
for key in range(model.nkey):
  mujoco.mj_resetDataKeyframe(model, data, key)
  mujoco.mj_forward(model, data)
  renderer.update_scene(data)
  media.show_image(renderer.render())

In [ ]:
vis = mujoco.MjvOption()
vis.geomgroup[2] = True
vis.geomgroup[3] = False
coll = mujoco.MjvOption()
coll.geomgroup[2] = False
coll.geomgroup[3] = True
coll.flags[mujoco.mjtVisFlag.mjVIS_CONVEXHULL] = True

camera = mujoco.MjvCamera()
mujoco.mjv_defaultFreeCamera(model, camera)
camera.distance = 6

duration = 10
fps = 30
nsteps = int(np.ceil(duration / model.opt.timestep))

# Set the desired control point.
if model.nkey > 0:
  mujoco.mj_resetDataKeyframe(model, data, 0)
  ctrl0 = data.ctrl.copy()
else:
  mujoco.mj_resetData(model, data)
  ctrl0 = np.mean(model.actuator_ctrlrange, axis=1)


In [ ]:
# print(data.joint(3))
# model.opt.timestep
nsteps

In [ ]:
'''
import mujoco
import numpy as np

# Определим параметры робота
wheel_base = 0.5  # расстояние между колесами в метрах

# Начальные условия
x, y, theta = 0.0, 0.0, 0.0  # начальные координаты и угол (в радианах)

# Целевая точка
target_x, target_y = 2.0, 2.0  # координаты цели

# Точность достижения цели
tolerance = 0.1  # 10 см

# Шаг симуляции (время шага)
dt = 0.01  # 10 мс

# Параметры контроллера
k_linear = 1.0  # коэффициент пропорциональности для линейной скорости
k_angular = 2.0  # коэффициент пропорциональности для угловой скорости

# Определим функцию для обновления одометрии
def update_odometry(data, x, y, theta, dt):
    linear_velocity = data.ctrl[0]  # линейная скорость
    angular_velocity = data.ctrl[1]  # угловая скорость (радиус поворота)

    # Вычисление изменения позиций
    delta_theta = angular_velocity * dt
    delta_x = linear_velocity * dt * np.cos(theta + delta_theta / 2)
    delta_y = linear_velocity * dt * np.sin(theta + delta_theta / 2)

    # Обновление позиции и угла
    x += delta_x
    y += delta_y
    theta += delta_theta

    # Нормализация угла
    theta = (theta + np.pi) % (2 * np.pi) - np.pi

    return x, y, theta

# Функция для расчета скоростей
def calculate_speeds(x, y, theta, target_x, target_y):
    # Вычисляем угловую ошибку
    angle_to_target = np.arctan2(target_y - y, target_x - x)
    angle_error = angle_to_target - theta

    # Нормализация угловой ошибки
    angle_error = (angle_error + np.pi) % (2 * np.pi) - np.pi

    # Пропорциональное управление угловой скоростью
    angular_velocity = k_angular * angle_error

    # Пропорциональное управление линейной скоростью
    distance_to_target = np.sqrt((target_x - x)**2 + (target_y - y)**2)
    linear_velocity = k_linear * distance_to_target

    # Ограничение скоростей
    max_linear_velocity = 0.2  # м/с
    max_angular_velocity = 1.0  # рад/с

    linear_velocity = np.clip(linear_velocity, 0, max_linear_velocity)
    angular_velocity = np.clip(angular_velocity, -max_angular_velocity, max_angular_velocity)

    return linear_velocity, angular_velocity

# Установим начальные управляющие воздействия
data.ctrl[0] = 1 # линейная скорость 0.1 м/с
data.ctrl[1] = 0.05  # угловая скорость 0.05 рад/с

# Количество шагов симуляции
frames = []

for i in tqdm(range(nsteps)):
    # Расчет управляющих воздействий
    linear_velocity, angular_velocity = calculate_speeds(x, y, theta, target_x, target_y)

    data.ctrl[0] = linear_velocity
    data.ctrl[1] = angular_velocity

    # Обновление состояния робота
    x, y, theta = update_odometry(data, x, y, theta, dt)

    # Проверка достижения цели
    distance_to_target = np.sqrt((target_x - x)**2 + (target_y - y)**2)
    if distance_to_target <= tolerance:
        # Остановка робота
        data.ctrl[0] = 0.0
        data.ctrl[1] = 0.0
        # print(f"Target reached at step {step}: x: {x:.3f}, y: {y:.3f}, theta: {theta:.3f}")
        break
    
    mujoco.mj_step(model, data)
    if len(frames) < data.time * fps:
        renderer.update_scene(data, camera, scene_option=vis)
        vispix = renderer.render().copy().astype(np.float32)
        frame = vispix.astype(np.uint8)
        frames.append(frame)
            
media.show_video(frames, fps=fps, loop=False)
'''


In [24]:
from math import *

class Robot:
    def __init__(self, model, data, vis, coll, camera):
        self.model = model
        self.data = data
        self.vis = vis
        self.coll = coll
        self.camera = camera
        self.x = 0
        self.y = 0
        self.tangent_angle = 0.0
        self.velocity = 0.0
        self.breadth = 0.0
        self.dt = 0.1
        self.treshold = 1
        
    
    def set_wheel_speeds(self, left_speed, right_speed):
        # self.data.joint('joint_right_wheel').qvel = np.array([right_speed])
        # self.data.joint('joint_left_wheel').qvel = np.array([left_speed])
        # print(left_speed, right_speed)
        data.ctrl[0] = 1 * (left_speed + right_speed) / 2.0

    
    def distance_to_point(self, x_goal, y_goal, x, y):
        return sqrt((x_goal - x) ** 2 + (y_goal - y) ** 2)
    
    def arc_radius(x1, y1, x2, y2, tangent_angle):
        chord_length = sqrt((x2 - x1)**2 + (y2 - y1)**2)
        chord_angle = atan2(y2 - y1, x2 - x1)
        next_angle = (chord_angle - tangent_angle) * 2
        radius = (chord_length / 2) * sin(chord_angle)
        # radius = chord_length / (2 * sin(pi - tangent_angle - abs(chord_angle)))
        # print(radius)
        data.ctrl[1] = -1 * radius
        return radius, next_angle
    
    def update_position(self, left_speed, right_speed, dt):
        wheel_base = self.breadth
        v = (left_speed + right_speed) / 2.0
        omega = (right_speed - left_speed) / wheel_base
        self.x += 1 * cos(self.tangent_angle) * dt
        self.y += 1 * sin(self.tangent_angle) * dt
        self.tangent_angle += omega * dt
        # print(f'x, y: {self.x, self.y}, v: {v}')
    
    def control_robot_to_point(self, x_goal, y_goal):
        duration = 10
        fps = 30
        nsteps = int(np.ceil(duration / model.opt.timestep))
        x = self.x
        y = self.y
        
        # Set the desired control point.
        if model.nkey > 0:
            mujoco.mj_resetDataKeyframe(model, data, 0)
            ctrl0 = data.ctrl.copy()
        else:
            mujoco.mj_resetData(model, data)
            ctrl0 = np.mean(model.actuator_ctrlrange, axis=1)

        frames = []
        
        for i in tqdm(range(nsteps)):
            # print(data.joint(1).qpos)
        # while self.distance_to_point(self, x_goal, y_goal, self.x, self.y) > self.threshold:
            distance = self.distance_to_point(self, x_goal, y_goal, self.x, self.y)    
            if distance > self.threshold:                         
                radius, next_angle = self.arc_radius(self.x, self.y, x_goal, y_goal, self.tangent_angle)
                # self.tangent_angle = next_angle
                chord_angle = atan2(y_goal - self.y, x_goal - self.x)
                angle_diff = chord_angle * 2
                # angle_diff = angle_to_goal - self.theta
                # omega = self.velocity / radius
                # omega = self.velocity / radius
                
                if angle_diff > pi:
                    angle_diff -= 2 * pi
                elif angle_diff < -pi:
                    angle_diff += 2 * pi
                
                if radius == 0:
                    v_left = self.velocity
                    v_right = self.velocity
                else:
                    # omega = self.velocity / radius
                    v_left = self.velocity * (radius + self.breadth/2) / radius
                    v_right = self.velocity * (radius - self.breadth/2) / radius
                # print(distance, radius, v_left, v_right)
                # v_left = v - omega * angle_diff
                # v_right = v + omega * angle_diff
                
                # self.set_wheel_speeds(self, v_left, v_right)
                self.set_wheel_speeds(self, v_left, v_right)
                data.ctrl[0] = 1
                # data.ctrl[1] = radius
                self.update_position(self, v_left, v_right, self.dt)

                '''  ### #### ###  '''
                # for i in tqdm(range(nsteps)):
                
                # print(v_left, v_right)
                # data.joint('joint_right_wheel').qvel = np.array([rv])
                # data.joint('joint_left_wheel').qvel = np.array([rl])
            else:
                self.set_wheel_speeds(self, 0, 0)


            
            mujoco.mj_step(model, data)
            if len(frames) < data.time * fps:
                renderer.update_scene(data, self.camera, scene_option=self.vis)
                vispix = renderer.render().copy().astype(np.float32)
                frame = vispix.astype(np.uint8)
                frames.append(frame)
            
        media.show_video(frames, fps=fps, loop=False)

        self.set_wheel_speeds(self, 0, 0)
        print("Reached the goal and stopped.")

robot = Robot
robot.__init__(robot, model, data, vis, coll, camera)
robot.velocity = 0.5
robot.breadth = 3.4
x_goal, y_goal = 7, -7
robot.x, robot.y = 0, 0 
robot.threshold = 2
robot.tangent_angle = 0
robot.control_robot_to_point(robot, x_goal, y_goal)


100%|██████████| 5000/5000 [00:02<00:00, 1754.30it/s]


This browser does not support the video tag.


Reached the goal and stopped.
